In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install bayesian-optimization

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
import pickle
import time
import sys

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.losses import mse
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-05-18 20:38:29.848546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 20:38:29.848616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-05-18 20:38:32.109239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 20:38:32.139508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 20:38:32.140038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-18 20:38:32.140473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared obj

# Samsung Webcam Data Pre-Processing

In [3]:
# Benign traffic

sw_benign = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/benign_traffic.csv', encoding = "utf-8", sep = ',' ) 
df_sw_benign = sw_benign.copy(deep=True)

columns = list(df_sw_benign.columns)
chosen_columns = []
for column in columns:
    if column.find('L5') != -1:
        chosen_columns.append(column)

df_sw_benign = pd.DataFrame(df_sw_benign, columns = chosen_columns)

# Bashlite

sw_bashlite_combo = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/gafgyt/combo.csv', encoding = "utf-8", sep = ',' ) 
df_sw_bashlite_combo = sw_bashlite_combo.copy(deep=True)
df_sw_bashlite_combo = pd.DataFrame(df_sw_bashlite_combo, columns = chosen_columns)
df_sw_bashlite_combo = df_sw_bashlite_combo.sample(frac=1)

sw_bashlite_junk = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/gafgyt/junk.csv', encoding = "utf-8", sep = ',' ) 
df_sw_bashlite_junk = sw_bashlite_junk.copy(deep=True)
df_sw_bashlite_junk = pd.DataFrame(df_sw_bashlite_junk, columns = chosen_columns)
df_sw_bashlite_junk = df_sw_bashlite_junk.sample(frac=1)

sw_bashlite_scan = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/gafgyt/scan.csv', encoding = "utf-8", sep = ',' ) 
df_sw_bashlite_scan = sw_bashlite_scan.copy(deep=True)
df_sw_bashlite_scan = pd.DataFrame(df_sw_bashlite_scan, columns = chosen_columns)
df_sw_bashlite_scan = df_sw_bashlite_scan.sample(frac=1)

sw_bashlite_udp = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/gafgyt/udp.csv', encoding = "utf-8", sep = ',' ) 
df_sw_bashlite_udp = sw_bashlite_udp.copy(deep=True)
df_sw_bashlite_udp = pd.DataFrame(df_sw_bashlite_udp, columns = chosen_columns)
df_sw_bashlite_udp = df_sw_bashlite_udp.sample(frac=1)

sw_bashlite_tcp = pd.read_csv('nbaiot/Samsung_SNH_1011_N_Webcam/gafgyt/tcp.csv', encoding = "utf-8", sep = ',' ) 
df_sw_bashlite_tcp = sw_bashlite_tcp.copy(deep=True)
df_sw_bashlite_tcp = pd.DataFrame(df_sw_bashlite_tcp, columns = chosen_columns)
df_sw_bashlite_tcp = df_sw_bashlite_tcp.sample(frac=1)

In [4]:
# Normalizing information

scaler = MinMaxScaler()

df_sw_bashlitecombo_norm = scaler.fit_transform(df_sw_bashlite_combo)
df_sw_bashlitejunk_norm = scaler.fit_transform(df_sw_bashlite_junk)
df_sw_bashlitescan_norm = scaler.fit_transform(df_sw_bashlite_scan)
df_sw_bashliteudp_norm = scaler.fit_transform(df_sw_bashlite_udp)
df_sw_bashlitetcp_norm = scaler.fit_transform(df_sw_bashlite_tcp)

In [5]:
# Bashlite attack labelization

label_bashlite_combo = list(np.full(df_sw_bashlitecombo_norm.shape[0], 5))
bashlitecombo_norm = pd.DataFrame(df_sw_bashlitecombo_norm)
bashlitecombo_norm['Label'] = label_bashlite_combo

label_bashlite_junk = list(np.full(df_sw_bashlitejunk_norm.shape[0], 6))
bashlitejunk_norm = pd.DataFrame(df_sw_bashlitejunk_norm)
bashlitejunk_norm['Label'] = label_bashlite_junk

label_bashlite_scan = list(np.full(df_sw_bashlitescan_norm.shape[0], 7))
bashlitescan_norm = pd.DataFrame(df_sw_bashlitescan_norm)
bashlitescan_norm['Label'] = label_bashlite_scan

label_bashlite_udp = list(np.full(df_sw_bashliteudp_norm.shape[0], 8))
bashliteudp_norm = pd.DataFrame(df_sw_bashliteudp_norm)
bashliteudp_norm['Label'] = label_bashlite_udp

label_bashlite_tcp = list(np.full(df_sw_bashlitetcp_norm.shape[0], 9))
bashlitetcp_norm = pd.DataFrame(df_sw_bashlitetcp_norm)
bashlitetcp_norm['Label'] = label_bashlite_tcp

# CNN - Attack Classification

## Model

In [6]:
batch_size = 50
number_features = 23
learning_rate = 0.008
epochs = 15

dict_params = { 'learning_rate': learning_rate, 'batch_size': int(32 * batch_size), 'epochs': int(epochs) }
pbounds = { 'learning_rate': (0.000001, 0.1), 'batch_size': (1, 4.001), 'epochs': (1, 100) }

In [7]:
# Train set

len_bashlite_combo_train = int(0.7 * len(bashlitecombo_norm))
X_train_bashlite_combo = bashlitecombo_norm[:len_bashlite_combo_train]

len_bashlite_junk_train = int(0.7 * len(bashlitejunk_norm))
X_train_bashlite_junk = bashlitejunk_norm[:len_bashlite_junk_train]

len_bashlite_scan_train = int(0.7 * len(bashlitescan_norm))
X_train_bashlite_scan = bashlitescan_norm[:len_bashlite_scan_train]

len_bashlite_udp_train = int(0.7 * len(bashliteudp_norm))
X_train_bashlite_udp = bashliteudp_norm[:len_bashlite_udp_train]

len_bashlite_tcp_train = int(0.7 * len(bashlitetcp_norm))
X_train_bashlite_tcp = bashlitetcp_norm[:len_bashlite_tcp_train]

np_train = np.concatenate([X_train_bashlite_combo, X_train_bashlite_junk, X_train_bashlite_scan, X_train_bashlite_udp,
                           X_train_bashlite_tcp])

df_train = pd.DataFrame(np_train)
label_train = df_train.pop(number_features)

X_train = df_train.to_numpy()
Y_train = label_train.to_numpy()

print(len(X_train))

226148


In [8]:
# Test set

len_bashlite_combo_test = len_bashlite_combo_train + int(0.15 * len(bashlitecombo_norm))
X_test_bashlite_combo = bashlitecombo_norm[len_bashlite_combo_train : len_bashlite_combo_test]

len_bashlite_junk_test = len_bashlite_junk_train + int(0.15 * len(bashlitejunk_norm))
X_test_bashlite_junk = bashlitejunk_norm[len_bashlite_junk_train : len_bashlite_junk_test]

len_bashlite_scan_test = len_bashlite_scan_train + int(0.15 * len(bashlitescan_norm))
X_test_bashlite_scan = bashlitescan_norm[len_bashlite_scan_train : len_bashlite_scan_test]

len_bashlite_udp_test = len_bashlite_udp_train + int(0.15 * len(bashliteudp_norm))
X_test_bashlite_udp = bashliteudp_norm[len_bashlite_udp_train : len_bashlite_udp_test]

len_bashlite_tcp_test = len_bashlite_tcp_train + int(0.15 * len(bashlitetcp_norm))
X_test_bashlite_tcp = bashlitetcp_norm[len_bashlite_tcp_train : len_bashlite_tcp_test]

np_test = np.concatenate([X_test_bashlite_combo, X_test_bashlite_junk, X_test_bashlite_scan, X_test_bashlite_udp,
                          X_test_bashlite_tcp])

df_test = pd.DataFrame(np_test)
label_test = df_test.pop(number_features)

X_test = df_test.to_numpy()
Y_test = label_test.to_numpy()

print(len(X_test))

48458


In [9]:
# Validation set

X_val_bashlite_combo = bashlitecombo_norm[len_bashlite_combo_test:]
X_val_bashlite_junk = bashlitejunk_norm[len_bashlite_junk_test:]
X_val_bashlite_scan = bashlitescan_norm[len_bashlite_scan_test:]
X_val_bashlite_udp = bashlitetcp_norm[len_bashlite_udp_test:]
X_val_bashlite_tcp = bashlitetcp_norm[len_bashlite_tcp_test:]

np_val = np.concatenate([X_val_bashlite_combo, X_val_bashlite_junk, X_val_bashlite_scan, X_val_bashlite_udp,
                         X_val_bashlite_tcp])

df_val = pd.DataFrame(np_test)
label_test = df_val.pop(number_features)

X_val = df_val.to_numpy()
Y_val = label_test.to_numpy()

In [10]:
X_train_CNN = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_CNN = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val_CNN = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

Y_train_CNN = Y_train
Y_test_CNN = Y_test
Y_val_CNN = Y_val

samples, feature, depth = X_train_CNN.shape

In [11]:
# CNN model with 1D convolutional layer

def CNN():
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform', input_shape=(feature, depth)))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [12]:
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

## Train

In [13]:
# Training step with Bayesian optimization

def training(X_train = X_train_CNN,
             Y_train = Y_train_CNN, 
             X_val = X_val_CNN, 
             Y_val = Y_val_CNN, 
             X_test = X_test_CNN, 
             Y_test = Y_test_CNN, 
             learning_rate = learning_rate, 
             epochs = epochs, 
             batch_size = batch_size,
             reduce_lr = reduce_lr):
    
    nadam = optimizers.Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
    
    model = CNN()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
    
    history = model.fit(X_train, Y_train, 
                        epochs = dict_params['epochs'], 
                        batch_size = dict_params['batch_size'], 
                        validation_data = (X_val, Y_val),
                        callbacks = [reduce_lr])
    
    scores = model.evaluate(X_test, Y_test)
    
    print('loss:', scores[0])
    print('accuracy:', scores[1])
    return scores[1]

In [14]:
# Bayesian optimization to choose the best hyperparameters

optimizer = BayesianOptimization(
    f = training,
    pbounds = pbounds,
    verbose = 2, 
    random_state = 1,
)

train_start = time.time()

optimizer.maximize(init_points = 5, n_iter = 5)

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

print(optimizer.max)

|   iter    |  target   | batch_... |  epochs   | learni... |
-------------------------------------------------------------
Train on 226148 samples, validate on 48458 samples
Epoch 1/15


2022-05-18 20:38:47.065201: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-18 20:38:47.094256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


225600/226148 [============================>.] - ETA: 0s - loss: 0.1878 - accuracy: 0.9550

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


226148/226148 [==============================] - 4s 20us/sample - loss: 0.1873 - accuracy: 0.9552 - val_loss: 0.0065 - val_accuracy: 0.9989 - lr: 0.0010
Epoch 2/15
226148/226148 [==============================] - 4s 17us/sample - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0032 - val_accuracy: 0.9993 - lr: 0.0010
Epoch 3/15
226148/226148 [==============================] - 4s 18us/sample - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0023 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 4/15
226148/226148 [==============================] - 4s 19us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0020 - val_accuracy: 0.9997 - lr: 0.0010
Epoch 5/15
226148/226148 [==============================] - 4s 19us/sample - loss: 8.0722e-04 - accuracy: 0.9998 - val_loss: 0.0017 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 6/15
226148/226148 [==============================] - 4s 19us/sample - loss: 6.4562e-04 - accuracy: 0.9999 - val_loss: 0.0014 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 7/15
226148/2

...

226148/226148 [==============================] - 8s 37us/sample - loss: 0.1897 - accuracy: 0.9572 - val_loss: 0.0073 - val_accuracy: 0.9988 - lr: 0.0010
Epoch 2/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0032 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 3/15
226148/226148 [==============================] - 7s 31us/sample - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.0024 - val_accuracy: 0.9997 - lr: 0.0010
Epoch 4/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0019 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 5/15
226148/226148 [==============================] - 7s 31us/sample - loss: 8.8003e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 6/15
226148/226148 [==============================] - 7s 31us/sample - loss: 7.0626e-04 - accuracy: 0.9999 - val_loss: 0.0016 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 7/15
226148/2

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


226148/226148 [==============================] - 8s 35us/sample - loss: 0.1758 - accuracy: 0.9617 - val_loss: 0.0060 - val_accuracy: 0.9990 - lr: 0.0010
Epoch 2/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0033 - accuracy: 0.9994 - val_loss: 0.0029 - val_accuracy: 0.9993 - lr: 0.0010
Epoch 3/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.0023 - val_accuracy: 0.9997 - lr: 0.0010
Epoch 4/15
226148/226148 [==============================] - 7s 31us/sample - loss: 9.8665e-04 - accuracy: 0.9998 - val_loss: 0.0019 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 5/15
226148/226148 [==============================] - 7s 31us/sample - loss: 7.1474e-04 - accuracy: 0.9999 - val_loss: 0.0016 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 6/15
226148/226148 [==============================] - 7s 30us/sample - loss: 5.8687e-04 - accuracy: 0.9999 - val_loss: 0.0015 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 7/15
2261

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


226148/226148 [==============================] - 8s 34us/sample - loss: 0.1549 - accuracy: 0.9678 - val_loss: 0.0150 - val_accuracy: 0.9962 - lr: 0.0010
Epoch 2/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0044 - val_accuracy: 0.9990 - lr: 0.0010
Epoch 3/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0029 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 4/15
226148/226148 [==============================] - 7s 30us/sample - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.0021 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 5/15
226148/226148 [==============================] - 7s 30us/sample - loss: 8.8694e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 6/15
226148/226148 [==============================] - 7s 31us/sample - loss: 6.7518e-04 - accuracy: 0.9999 - val_loss: 0.0015 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 7/15
226148/2

In [15]:
# Training step with the best hyperparameters

nadam = optimizers.Nadam(learning_rate = optimizer.max['params']['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)

model = CNN()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])

train_start = time.time()

history = model.fit(X_train_CNN, Y_train_CNN, 
                    epochs = int(optimizer.max['params']['epochs']), 
                    batch_size = int(32 * optimizer.max['params']['batch_size']), 
                    validation_data = (X_val_CNN, Y_val_CNN),
                    callbacks = [reduce_lr])

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

Train on 226148 samples, validate on 48458 samples
Epoch 1/15
226148/226148 [==============================] - 39s 173us/sample - loss: 0.0103 - accuracy: 0.9978 - val_loss: 0.0012 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 2/15
226148/226148 [==============================] - 37s 165us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 9.1687e-04 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 3/15
226148/226148 [==============================] - 39s 172us/sample - loss: 8.4871e-04 - accuracy: 0.9998 - val_loss: 9.4183e-04 - val_accuracy: 0.9999 - lr: 0.0010
Epoch 4/15
226148/226148 [==============================] - 39s 173us/sample - loss: 3.7667e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 5/15
226148/226148 [==============================] - 39s 174us/sample - loss: 6.1447e-04 - accuracy: 0.9999 - val_loss: 6.4154e-04 - val_accuracy: 0.9999 - lr: 0.0010
Epoch 6/15
226148/226148 [==============================] - 40s 175us/sample - loss: 3.6505e-04

## Test

In [16]:
# Testing step

test_start = time.time()

Y_pred = model.predict(X_test_CNN)

test_end = time.time()
test_time = test_end - test_start
print("Testing time:", test_time)

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Testing time: 7.389723777770996


In [17]:
Y_pred_CNN = np.argmax(Y_pred, axis = 1)
Y_true_CNN = Y_test_CNN.astype(int)

## Metrics

In [18]:
labels = ['Bashlite_Combo','Bashlite_Junk','Bashlite_Scan','Bashlite_Udp', 'Bashlite_Tcp']

In [19]:
# Multi classification metrics

acc = accuracy_score(Y_true_CNN, Y_pred_CNN) 
f1 = f1_score(Y_true_CNN, Y_pred_CNN, average = 'weighted')
pre = precision_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted')
recall = recall_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

In [20]:
sys.stdout = open("Results/samsung_webcam_cnn.txt", "a")

print(" ==== Test " + str(number_features) + " features ====")
print("Training time:" + str(train_time))
print("Testing time:" + str(test_time))
print("Accuracy:" + str(acc))
print("F1-score:" + str(f1))
print("Precision:" + str(pre))
print("Recall:" + str(recall))
print(classification_report(Y_true_CNN,Y_pred_CNN, target_names = labels))

## Naive Bayes Model

In [21]:
# Model
nb = GaussianNB()

# Train
train_nb_start = time.time()

nb.fit(X_train, Y_train)

train_nb_end = time.time()
train_nb_time = train_nb_end - train_nb_start

# Test
test_nb_start = time.time()

Y_pred_nb = nb.predict(X_test)

test_nb_end = time.time()
test_nb_time = test_nb_end - test_nb_start

# Metrics
acc_nb = accuracy_score(Y_test, Y_pred_nb) 
f1_nb = f1_score(Y_test, Y_pred_nb, average = 'weighted')
pre_nb = precision_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted')
recall_nb = recall_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== Naive Bayes " + str(number_features) + " features ====")
print("Training time:" + str(train_nb_time))
print("Testing time:" + str(test_nb_time))
print("Accuracy:" + str(acc_nb))
print("F1-score:" + str(f1_nb))
print("Precision:" + str(pre_nb))
print("Recall:" + str(recall_nb))
print(classification_report(Y_test,Y_pred_nb, target_names = labels))

## KNN Model

In [22]:
# Model
knn = KNeighborsClassifier(n_neighbors = 50)

# Train
train_knn_start = time.time()

knn.fit(X_train, Y_train)

train_knn_end = time.time()
train_knn_time = train_knn_end - train_knn_start

# Test
test_knn_start = time.time()

Y_pred_knn = knn.predict(X_test)

test_knn_end = time.time()
test_knn_time = test_knn_end - test_knn_start

# Metrics
acc_knn = accuracy_score(Y_test, Y_pred_knn) 
f1_knn = f1_score(Y_test, Y_pred_knn, average = 'weighted')
pre_knn = precision_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted')
recall_knn = recall_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== KNN " + str(number_features) + " features ====")
print("Training time:" + str(train_knn_time))
print("Testing time:" + str(test_knn_time))
print("Accuracy:" + str(acc_knn))
print("F1-score:" + str(f1_knn))
print("Precision:" + str(pre_knn))
print("Recall:" + str(recall_knn))
print(classification_report(Y_test,Y_pred_knn, target_names = labels))